# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fort bragg,35.1390,-79.0060,42.82,63,0,8.05,US,1704726703
1,1,ilulissat,69.2167,-51.1000,15.82,67,75,14.97,GL,1704726703
2,2,itamaraca,-7.7478,-34.8256,89.65,60,7,11.43,BR,1704726703
3,3,adamstown,-25.0660,-130.1015,73.98,82,100,12.50,PN,1704726704
4,4,ushuaia,-54.8000,-68.3000,49.62,89,100,6.35,AR,1704726704


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng",
                                       "Lat",
                                       geo = True,
                                       color = "City",
                                       alpha = .6,
                                       size = "Humidity", 
                                       tiles = "OSM",
                                       hover_cols=["City", "Country", "Humidity", "Hotel Name"]
                                      
                                      )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 32)
                                       & (city_data_df["Max Temp"] > 17)
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
166,166,nili,33.7218,66.1302,31.12,58,2,4.12,AF,1704726794
384,384,mountain home,43.1330,-115.6912,18.88,86,0,0.00,US,1704726861


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
166,nili,AF,33.7218,66.1302,58,
384,mountain home,US,43.1330,-115.6912,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accomodation.hotel", 
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nili - nearest hotel: No hotel found
mountain home - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
166,nili,AF,33.7218,66.1302,58,No hotel found
384,mountain home,US,43.1330,-115.6912,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat",
                                   geo = True,
                                   color = "City",
                                   alpha = .6,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["City", "Country", "Hotel Name", "Humidity"],
                                   frame_width = 600, 
                                   frame_height = 400,
                                   aspect = 1,
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)